In [1]:
# %load create_train_data.py
# This program intends to make the car console interface to generate car driving training data.
import numpy as np
import cv2
import time

brake_path = 'brake_frames/brake'
gas_path = 'gas_frames/acceleration'
gear_path = 'gear_frames/gear'

out_imgpath = 'indian_dataset/data.txt'
f = open(out_imgpath, "w")

smoothed_angle = 0
degrees = 1
gas_pedal = 0
brake_pedal = 0
gear = 0
gearShift = 0
frame = 1
writeTime = -1
frameRate = []
keepUpArrowPressed = False

# in training mode, animation should be switched off for syncing.
animate = True

#img = cv2.imread('steering_wheel_image.jpg',0)
#rows,cols = img.shape()
from PIL import Image

# open method used to open different extension image file
img = np.array(Image.open(r"unnamed.jpeg"))
rows, cols, channel= img.shape


# To display neutral gear
gear_img = cv2.imread(gear_path + " " + str(1).zfill(2)+".jpg")
cv2.imshow('Gear-Transmission', gear_img)


start = int(round(time.time() * 1000))

while(True):

	# to return full key code so that arrow keys also can be detected.
	k = cv2.waitKeyEx(1)

	# Scan Codes for arrow keys
	# 2490368 up - for gas_pedal
	# 2621440 down - for break
	# 2424832 left - to turn left
	# 2555904 right - to turn right
	# a = to shift gear up
	# z = to shift gear down
	# n = to shift to neutral (from all gears)

	if (k==2490368):
		print('up')
		if (gas_pedal < 5): # max value of gas = 5
			gas_pedal += 0.5
			keepUpArrowPressed = True
		print('gas = ' + str(gas_pedal))
	elif (k==2621440):
		print('down')
		if (brake_pedal < 5): # max value of break = 5
			brake_pedal += 0.5
			keepUpArrowPressed = False
		print('break = ' + str(brake_pedal))
	elif (k==2555904):
		print('right')
		degrees += 3
		print('Angle = ' + str(degrees))
	elif (k==2424832):
		print('left')
		degrees -= 3
		print('Angle = ' + str(degrees))
	elif (k == ord('a')):
		print("Gear up")
		if (gear < 5):
			gear += 1
			gearShift = 1
		print('Gear = ' + str(gear))
	elif (k == ord('z')):
		print("Gear down")
		if (gear > 0):
			gear -= 1
			gearShift = -1
		print('Gear = ' + str(gear))
	elif (k == ord('g')):
		print("Neutral Gear")
		if (gear != 0):
			gearShift = -2
		print('Gear = ' + str(gear))
	else:
		if (gas_pedal > 0.01 and  not keepUpArrowPressed): # min value of gas = 0
			gas_pedal -= 0.005
		if (brake_pedal > 0.01): # min value of break = 0
			brake_pedal -= 0.005

		gearShift = 0

	
	smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
	M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
	dst = cv2.warpAffine(img,M,(cols,rows))
	cv2.imshow("steering wheel", dst)

	if (animate):
		# To show the braking animation
		brake_img = cv2.imread(brake_path + " " + str(int(brake_pedal*2+1)).zfill(2)+".jpg")
		brake_img_small = cv2.resize(brake_img, (0,0), fx=0.5, fy=0.5) 
		cv2.imshow('Brake Pedal', brake_img_small)

		# To show the gal pedal animation
		accel_img = cv2.imread(gas_path + " " + str(int(gas_pedal*2+1)).zfill(2)+".jpg")
		accel_img_small = cv2.resize(accel_img, (0,0), fx=0.52, fy=0.52) 
		cv2.imshow('Acceleration Pedal', accel_img_small)

		# To show the gear animation: Frames are mapped as below.
		# 0-10: 1st gear, 11-20: 2nd gear, 21-30: 3rd gear, 
		# 30-37: 4th gear, 37-45: 5th gear, 45-50: 5th to neutral.

		# Frames correspond to Neutral, 1st, 2nd, 3rd, 4th and 5th gears
		gear_frame_map = [1, 12, 22, 30, 37, 45] 
		# Frames correspond to neutral between gears
		neutral_frame_map = [16, 26, 33, 42, 50]

		if (gearShift != 0):

			# To animate shift from any gear to neutral
			if (gearShift == -2):
				init_frame = gear_frame_map [gear]
				end_frame = neutral_frame_map [gear-1]

				for i in range(init_frame, end_frame, 1):
					gear_img = cv2.imread(gear_path + " " + str(i).zfill(2)+".jpg")
					cv2.imshow('Gear-Transmission', gear_img)
					cv2.waitKey(1)

				gear = 0

			else: # when gearShift = -1 and +1 (gear up or down)
				init_frame = gear_frame_map [gear + gearShift*-1]
				end_frame = gear_frame_map [gear]
				print("init_frame = " + str(init_frame) + " end_frame = " + str(end_frame))

				for i in range(init_frame, end_frame, gearShift):
					# File read should confirm with the video to jpg frame extractor software
					gear_img = cv2.imread(gear_path + " " + str(i).zfill(2)+".jpg")
					# gear_img_small = cv2.resize(gear_img, (0,0), fx=0.5, fy=0.5) 
					cv2.imshow('Gear-Transmission', gear_img)
					cv2.waitKey(1)


	print('gas = ' + str(gas_pedal))
	print('break = ' + str(brake_pedal))

	# 1 driving second = 30 frames (extracted). We need to sync with this frame rate to find corresponding params of each frame.
	# Hence we will write the parameter values for acceleration, brake, gear and steering every 33 milliseconds. To average out 
	# around 33 ms, we need to tune the write frequency cut off, considering the running time of each while loop.
	now = int(round(time.time() * 1000000))
	
	if (now - writeTime >= 30001):
		print("time = " + str(now - writeTime))
		if (now - writeTime < 100000):
			frameRate.append(now - writeTime)

		# print(gas_pedal)
		print('gas = ' + str(gas_pedal))
		print('break = ' + str(brake_pedal))
		f.write(str(frame) + " " + str(degrees) + " " + str(round(gas_pedal, 2)) + " " + str(round(brake_pedal, 2)) + " " + str(gear) + '\n')
		writeTime = int(round(time.time() * 1000000))
		# to denote the frame of the input video
		frame += 1

	# To exit interface, press 'q'
	if (k == ord('q')):
		break


end = int(round(time.time() * 1000))

print("\n\nIdeal average write rate should be 33.33")
print("Average Write Rate = " + str(round(sum(frameRate)/ (1000*len(frameRate)), 2)))
print("Total Running Time = " + str(float((end-start)/1000)) + " seconds." + "Number of Frames in data.txt should be " + str(int((end-start)*30/1000)))

f.close()

gas = 0
break = 0
time = 1642681261714658
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32938
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 35962
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 40001
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 43924
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 42870
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33900
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30918
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 41045
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 40036
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 34689
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33111
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32906
gas = 0
break = 0
gas = 0
break = 0
gas = 0
br

gas = 0
break = 0
time = 49148
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32322
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30032
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 58535
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30011
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30024
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 46535
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33166
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 39993
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 50198
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 42856
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 43890
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 

gas = 0
break = 0
time = 39736
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 41777
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 42860
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30949
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 31916
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 31928
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32899
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33910
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 42886
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33917
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 33901
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32912
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32913
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32916
gas = 0
break = 0
gas = 0
break = 0
gas = 0


gas = 0
break = 0
gas = 0
break = 0
time = 30941
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 38948
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 36417
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 36541
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 34581
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30915
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32920
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 41611
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 43914
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 35692
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 42470
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 34870
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 48218
gas = 0
break = 0
gas = 0
break = 0
gas

gas = 0
break = 0
time = 33908
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 36903
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 35902
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32944
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 38045
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30035
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 41318
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 30919
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 34041
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 39997
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 39968
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 37454
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32877
gas = 0
break = 0
gas = 0
break = 0
gas = 0
break = 0
time = 32914
gas = 0
break = 0
gas = 0


...

up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.795000000000096
gas = 5.395000000000016
break = 2.790000000000096
time = 44174
gas = 5.395000000000016
break = 2.790000000000096
up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.790000000000096
gas = 5.395000000000016
break = 2.785000000000096
up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.785000000000096
time = 31117
gas = 5.395000000000016
break = 2.785000000000096
gas = 5.395000000000016
break = 2.780000000000096
up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.780000000000096
time = 30033
gas = 5.395000000000016
break = 2.780000000000096
gas = 5.395000000000016
break = 2.7750000000000963
gas = 5.395000000000016
break = 2.7700000000000964
up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.7700000000000964
time = 40000
gas = 5.395000000000016
break = 2.7700000000000964
gas = 5.395000000000016
break = 2.7650000000000965
up
gas = 5.395000000000016
gas = 5.395000000000016
break =

left
Angle = 259
gas = 5.395000000000016
break = 2.3800000000001047
time = 38612
gas = 5.395000000000016
break = 2.3800000000001047
gas = 5.395000000000016
break = 2.375000000000105
left
Angle = 256
gas = 5.395000000000016
break = 2.375000000000105
gas = 5.395000000000016
break = 2.370000000000105
time = 41055
gas = 5.395000000000016
break = 2.370000000000105
gas = 5.395000000000016
break = 2.365000000000105
left
Angle = 253
gas = 5.395000000000016
break = 2.365000000000105
gas = 5.395000000000016
break = 2.360000000000105
time = 30031
gas = 5.395000000000016
break = 2.360000000000105
up
gas = 5.395000000000016
gas = 5.395000000000016
break = 2.360000000000105
gas = 5.395000000000016
break = 2.3550000000001052
gas = 5.395000000000016
break = 2.3500000000001053
time = 39995
gas = 5.395000000000016
break = 2.3500000000001053
gas = 5.395000000000016
break = 2.3450000000001054
gas = 5.395000000000016
break = 2.3400000000001056
gas = 5.395000000000016
break = 2.3350000000001057
time = 40001

gas = 5.395000000000016
break = 1.895000000000115
gas = 5.395000000000016
break = 1.8900000000001151
gas = 5.395000000000016
break = 1.8850000000001153
time = 43839
gas = 5.395000000000016
break = 1.8850000000001153
gas = 5.395000000000016
break = 1.8800000000001154
right
Angle = 259
gas = 5.395000000000016
break = 1.8800000000001154
gas = 5.395000000000016
break = 1.8750000000001155
time = 36120
gas = 5.395000000000016
break = 1.8750000000001155
right
Angle = 262
gas = 5.395000000000016
break = 1.8750000000001155
gas = 5.395000000000016
break = 1.8700000000001156
right
Angle = 265
gas = 5.395000000000016
break = 1.8700000000001156
time = 39971
gas = 5.395000000000016
break = 1.8700000000001156
gas = 5.395000000000016
break = 1.8650000000001157
right
Angle = 268
gas = 5.395000000000016
break = 1.8650000000001157
gas = 5.395000000000016
break = 1.8600000000001158
time = 39999
gas = 5.395000000000016
break = 1.8600000000001158
gas = 5.395000000000016
break = 1.855000000000116
right
Angle

gas = 5.395000000000016
break = 1.475000000000124
gas = 5.395000000000016
break = 1.470000000000124
time = 31920
gas = 5.395000000000016
break = 1.470000000000124
gas = 5.395000000000016
break = 1.4650000000001242
gas = 5.395000000000016
break = 1.4600000000001243
gas = 5.395000000000016
break = 1.4550000000001244
time = 38946
gas = 5.395000000000016
break = 1.4550000000001244
gas = 5.395000000000016
break = 1.4500000000001245
gas = 5.395000000000016
break = 1.4450000000001246
gas = 5.395000000000016
break = 1.4400000000001247
time = 39999
gas = 5.395000000000016
break = 1.4400000000001247
gas = 5.395000000000016
break = 1.4350000000001248
gas = 5.395000000000016
break = 1.430000000000125
time = 30321
gas = 5.395000000000016
break = 1.430000000000125
gas = 5.395000000000016
break = 1.425000000000125
gas = 5.395000000000016
break = 1.4200000000001252
gas = 5.395000000000016
break = 1.4150000000001253
time = 41872
gas = 5.395000000000016
break = 1.4150000000001253
gas = 5.395000000000016

gas = 5.395000000000016
break = 0.910000000000134
gas = 5.395000000000016
break = 0.905000000000134
time = 44390
gas = 5.395000000000016
break = 0.905000000000134
gas = 5.395000000000016
break = 0.900000000000134
gas = 5.395000000000016
break = 0.895000000000134
gas = 5.395000000000016
break = 0.890000000000134
time = 45001
gas = 5.395000000000016
break = 0.890000000000134
gas = 5.395000000000016
break = 0.885000000000134
gas = 5.395000000000016
break = 0.880000000000134
gas = 5.395000000000016
break = 0.875000000000134
time = 46354
gas = 5.395000000000016
break = 0.875000000000134
gas = 5.395000000000016
break = 0.870000000000134
gas = 5.395000000000016
break = 0.865000000000134
time = 33908
gas = 5.395000000000016
break = 0.865000000000134
gas = 5.395000000000016
break = 0.860000000000134
gas = 5.395000000000016
break = 0.855000000000134
gas = 5.395000000000016
break = 0.850000000000134
time = 46066
gas = 5.395000000000016
break = 0.850000000000134
gas = 5.395000000000016
break = 0.8

In [10]:
# %load gen_optical_flow_traindata.py
import numpy as np
import cv2
import scipy.misc
import math
import imageio
from scipy import signal

cap = cv2.VideoCapture('slow.flv')

#read data.txt
xs = []
ys = []
accels = []
with open("indian_dataset/indian_dataset/data.txt") as f:
    for line in f:
        xs.append("indian_dataset/indian_dataset/circuit2_x264.mp4 " + str(line.split()[0]).zfill(5) + ".jpg")
        ys.append(float(line.split()[1]))



out_imgpath = 'indian_dataset/optFlow.txt'
optF = open(out_imgpath, "w")
lk_params = dict( winSize = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                              10, 0.03))
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Function to calculate distance
def distance(x1 , y1 , x2 , y2):

    # Calculating distance
    return math.sqrt(math.pow(x2 - x1, 2) +
                math.pow(y2 - y1, 2) * 1.0)


# Take first frame and find corners in it
frame = imageio.imread(xs[0])
previous_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_frame, mask = None, **feature_params)
p0 = cv2.goodFeaturesToTrack(previous_frame, 100, 0.01,10)

frameNum = 0

while(frameNum < len(xs)):

    frame = cv2.imread(xs[frameNum])
    present_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    mask = np.zeros_like(frame)
    p0 = cv2.goodFeaturesToTrack(present_frame, mask = None, **feature_params)
    p0 = cv2.goodFeaturesToTrack(present_frame, 100, 0.01,10)


    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_frame, present_frame, p0, nextPts=None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    distSum = 0
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        dist = distance (a, b, c, d)
        distSum += dist

    # When number of detected corners (good features) are less than 10 then distance becomes erroneous.
    if (good_old.shape[0] < 10):
        opticalParam = prev_opticalParam
    else:
        opticalParam = distSum/(math.pow(good_old.shape[0], 2)+math.pow(ys[frameNum], 2)+1)*100

    optF.write(str(opticalParam) + '\n')

    prev_opticalParam = opticalParam

    # Now update the previous frame and previous points
    previous_frame = present_frame.copy()

    frameNum += 1

cv2.destroyAllWindows()
optF.close()



In [8]:
# %load train_accel.py
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import scipy
import cv2

# standard step - reset computation graphs
tf.reset_default_graph()

saveDirectory = './save'
L2NormConst = 0.001

import model_accel
all_vars = tf.trainable_variables()
# model_accel_vars = [k for k in all_vars if 'accel' in k.name]
print(model_accel.y_accel_.shape)
print(model_accel.y_accel.shape)
loss_accel = tf.reduce_mean(tf.square(tf.subtract(model_accel.y_accel_, model_accel.y_accel))) #+ tf.add_n([tf.nn.l2_loss(v) for v in all_vars]) * L2NormConst
train_step_accel = tf.train.AdadeltaOptimizer(1., 0.95, 1e-6).minimize(loss_accel) # These are default parameters to the Adadelta optimizer
# train_step_accel = tf.train.AdamOptimizer().minimize(loss_accel)
sess_accel = tf.Session()
sess_accel.run(tf.global_variables_initializer())
saver_accel = tf.train.Saver(all_vars)


# create a summary to monitor cost tensor
tf.summary.scalar("loss_accel", loss_accel)

# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 50
batch_size = 80

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):

    xs, ys, accels, brakes = driving_data.LoadTrainBatch(batch_size, False)

    train_step_accel.run(feed_dict={model_accel.x_accel: xs, model_accel.y_accel_: accels, model_accel.keep_prob_accel: 0.5, model_accel.keep_prob_accel_conv: 0.25}, session = sess_accel)

    if i % 10 == 0:
      xs, ys, accels, brakes = driving_data.LoadValBatch(batch_size, False)
      loss_value_accel = loss_accel.eval(feed_dict={model_accel.x_accel:xs, model_accel.y_accel_: accels, model_accel.keep_prob_accel: 1.0, model_accel.keep_prob_accel_conv: 1.0}, session = sess_accel)
      print("Epoch: %d, Step: %d, Accel Loss: %g " % (epoch, epoch * batch_size + i, loss_value_accel))

    if i % batch_size == 0:
      if not os.path.exists(saveDirectory):
        os.makedirs(saveDirectory)
      accel_checkpoint_path = os.path.join(saveDirectory, "model_accel.ckpt")
      filename_accel = saver_accel.save(sess_accel, accel_checkpoint_path)

  print("Model saved in file: %s" % filename_accel)


### To predict the output based on the above trained model###
xs = []
dataPath = "indian_dataset/"
fileNamePrefix = "circuit2_x264.mp4 "

with open(dataPath+"data.txt") as f:
    for line in f:
        xs.append(dataPath + fileNamePrefix + str(int(line.split()[0])).zfill(5)+".jpg")

i = 0
while(cv2.waitKey(10) != ord('q')):

    full_image = scipy.misc.imread(xs[i], mode="RGB")
    
    image = scipy.misc.imresize(full_image[-150:], [112, 112]) / 255.0
    print(image.shape)
    print(model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0}))
    with g_accel.as_default():
        acceleration = model_accel.y_accel.eval(feed_dict={model_accel.x_accel: [image], model_accel.keep_prob_accel: 1.0, model_accel.keep_prob_accel_conv: 1.0}, session = sess_accel)[0][0]
        print(i,acceleration * 180.0 / scipy.pi)
        i += 1


#To Visualize CNN Layers to better interpretability. Base code obtained from:
#https://medium.com/@awjuliani/visualizing-neural-network-layer-activation-tensorflow-tutorial-d45f8bf7bbc4
def getActivations(layer,stimuli, filename, steer, acceleration):
    units = sess.run(layer,feed_dict={model.x:np.reshape(stimuli,[-1, 136, 240, 3],order='F'), model.y1_: np.reshape(steer, [-1, 1],order='F'), model.y2_: acceleration, model.keep_prob:1.0})
    plotNNFilter(units, filename)


def plotNNFilter(units, filename):
    filters = units.shape[3]
    plt.figure(1, figsize=(20,20))
    n_columns = 6
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="gray")
    plt.savefig(filename)

xs = []
ys = []
accels = []
with open("indian_dataset/data.txt") as f:
    for line in f:
        xs.append("indian_dataset/circuit2_x264.mp4 " + str(line.split()[0]).zfill(5) + ".jpg")
        ys.append(float(line.split()[1]) * scipy.pi / 180)
        accels.append(float(line.split()[2]))

frameNum = 4805
full_image = scipy.misc.imread(xs[frameNum], mode="RGB")
image = scipy.misc.imresize(full_image[-150:], [136, 240]) / 255.0
cv2.imshow("Visualize CNN: input image", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
cv2.imshow("Visualize CNN: input image", image)

getActivations(model_steer.h_conv1,image, 'cnn-depict-conv1.jpg', ys[frameNum], accels[frameNum])
getActivations(model_steer.h_conv2,image, 'cnn-depict-conv2.jpg', ys[frameNum], accels[frameNum])
getActivations(model_steer.h_conv3,image, 'cnn-depict-conv3.jpg', ys[frameNum], accels[frameNum])
getActivations(model_steer.h_conv4,image, 'cnn-depict-conv4.jpg', ys[frameNum], accels[frameNum])
getActivations(model_steer.h_conv5,image, 'cnn-depict-conv5.jpg', ys[frameNum], accels[frameNum])
getActivations(model_steer.h_conv6,image, 'cnn-depict-conv6.jpg', ys[frameNum], accels[frameNum])

sess_accel.close()

FileNotFoundError: [Errno 2] No such file or directory: 'indian_dataset/corr/optFlow.txt'